In [13]:
from ultralytics import YOLO
import cv2

model = YOLO('yolov8n.pt')

In [3]:
image = cv2.imread('ships_3.jpg')

results = model(image)

# Инициализация счетчика кораблей
ships_count = 0

# Перебираем все обнаруженные объекты
for result in results:
    for box in result.boxes:
        class_id = int(box.cls)
        if class_id == 8:  # Нужный класс в COCO
            ships_count += 1

# Визуализация результатов с bounding boxes
annotated_image = results[0].plot()

# Добавляем текст с количеством кораблей
text = f'SHIPS Count: {ships_count}'
cv2.putText(annotated_image, text, (850, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (180, 20, 220), 2)

# Показываем результат 
cv2.imshow('Ships Detection', annotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Выводим количество
#print(f"Найдено кораблей: {ships_count}")


0: 448x640 3 boats, 104.0ms
Speed: 6.0ms preprocess, 104.0ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)


In [ ]:
%tb
from flask import Flask, request, jsonify, send_from_directory
from ultralytics import YOLO
import cv2
import tempfile
import os
from flask_cors import CORS

import shutil

# Путь к папке временных файлов
TEMP_DIR = "temporary_folder"

# Очистка папки перед запуском (опционально)
shutil.rmtree(TEMP_DIR, ignore_errors=True)
os.makedirs(TEMP_DIR, exist_ok=True)

app = Flask(__name__)
CORS(app)  # Разрешить запросы с любых доменов
model = YOLO('yolov8n.pt')

@app.route('/')
def home():
    return "Succsess"

@app.route('/process', methods=['POST'])
def process_video():
    try:
        if 'video' not in request.files:
            return jsonify({'error': 'No video uploaded'}), 400

        video_file = request.files['video']
        if video_file.filename == '':
            return jsonify({'error': 'Empty filename'}), 400

        # Сохраняем входное видео
        input_path = os.path.join(TEMP_DIR, "input_video.mp4")
        video_file.save(input_path)

        # Инициализация видео-потоков
        cap = cv2.VideoCapture(input_path)
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Выходной файл
        output_filename = "output_video.mp4"
        output_path = os.path.join(TEMP_DIR, output_filename)

        fourcc = cv2.VideoWriter_fourcc(*'avc1')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        counts = []

        # Обработка кадров
        try:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                results = model(frame, classes=[8])
                count = len(results[0].boxes)
                counts.append(count)

                annotated_frame = results[0].plot()
                out.write(annotated_frame)
        finally:
            # Гарантированное освобождение ресурсов
            cap.release()
            out.release()

        return jsonify({
            'video_url': f'/download/{output_filename}',
            'counts': counts
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500


@app.route('/download/<filename>')
def download_file(filename):
    return send_from_directory(
        TEMP_DIR,
        filename,
        mimetype='video/mp4',  # Явно указываем MIME-тип
        as_attachment=False
    )

if __name__ == '__main__':
    app.run()

KeyboardInterrupt: 

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/May/2025 13:37:36] "OPTIONS /process HTTP/1.1" 200 -



0: 384x640 2 boats, 121.0ms
Speed: 5.0ms preprocess, 121.0ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 boats, 102.0ms
Speed: 5.0ms preprocess, 102.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 boats, 107.0ms
Speed: 3.0ms preprocess, 107.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 boats, 110.0ms
Speed: 3.0ms preprocess, 110.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 boats, 108.0ms
Speed: 3.0ms preprocess, 108.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 boats, 112.1ms
Speed: 4.0ms preprocess, 112.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 boats, 111.0ms
Speed: 4.0ms preprocess, 111.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 boats, 100.0ms
Speed: 2.0ms preprocess, 100.0ms inference, 1.0ms postprocess per image at shape (

127.0.0.1 - - [03/May/2025 13:37:51] "POST /process HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2025 13:37:54] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [03/May/2025 13:37:54] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [03/May/2025 13:37:54] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [03/May/2025 13:37:54] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [03/May/2025 13:37:54] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [03/May/2025 13:37:55] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [03/May/2025 13:37:55] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [03/May/2025 13:38:03] "OPTIONS /process HTTP/1.1" 200 -



0: 384x640 2 boats, 104.0ms
Speed: 3.0ms preprocess, 104.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 boats, 101.0ms
Speed: 5.0ms preprocess, 101.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 boats, 96.0ms
Speed: 2.0ms preprocess, 96.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 boats, 94.0ms
Speed: 3.0ms preprocess, 94.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 boats, 92.0ms
Speed: 2.0ms preprocess, 92.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 boats, 87.0ms
Speed: 3.0ms preprocess, 87.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 boats, 88.0ms
Speed: 2.0ms preprocess, 88.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 boats, 88.0ms
Speed: 2.0ms preprocess, 88.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 6

127.0.0.1 - - [03/May/2025 13:38:18] "POST /process HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2025 13:39:00] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [03/May/2025 13:39:00] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [03/May/2025 13:39:00] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [03/May/2025 13:39:00] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [03/May/2025 13:39:58] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [03/May/2025 13:39:59] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [03/May/2025 13:39:59] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [03/May/2025 13:43:29] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [03/May/2025 13:43:29] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [03/May/2025 13:43:29] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [03/May/2025 13:43:29] "GET /download/output_video.mp4 HTTP/1.1" 206


0: 384x640 2 boats, 157.0ms
Speed: 7.0ms preprocess, 157.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 boats, 159.0ms
Speed: 6.0ms preprocess, 159.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 boats, 140.0ms
Speed: 4.0ms preprocess, 140.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 boats, 158.0ms
Speed: 6.0ms preprocess, 158.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 boats, 141.0ms
Speed: 4.0ms preprocess, 141.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 boats, 133.0ms
Speed: 5.0ms preprocess, 133.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 boats, 139.0ms
Speed: 3.0ms preprocess, 139.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 boats, 147.1ms
Speed: 3.0ms preprocess, 147.1ms inference, 2.0ms postprocess per image at shape (

127.0.0.1 - - [03/May/2025 13:44:06] "POST /process HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2025 13:44:06] "GET /download/output_video.mp4 HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2025 13:44:06] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [03/May/2025 13:44:06] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [03/May/2025 13:44:07] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [03/May/2025 13:44:07] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [03/May/2025 13:44:07] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [03/May/2025 13:44:07] "GET /download/output_video.mp4 HTTP/1.1" 206 -


In [4]:
!pip install flask_cors

  Obtaining dependency information for flask_cors from https://files.pythonhosted.org/packages/85/61/4aea5fb55be1b6f95e604627dc6c50c47d693e39cab2ac086ee0155a0abd/flask_cors-5.0.1-py3-none-any.whl.metadata
